In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd

<IPython.core.display.Javascript object>

In [3]:
beds = pd.read_csv(
    "../beds.csv",
    names=[
        "country",
        "critical_beds_per_100k",
        "capacity_active_infection_percent",
        "capacity_new_infections_per_day_per_1000",
        "year",
        "source",
    ],
)

<IPython.core.display.Javascript object>

In [4]:
beds[beds.country.duplicated(keep="last")]

,country,critical_beds_per_100k,capacity_active_infection_percent,capacity_new_infections_per_day_per_1000,year,source
50,Spain,9.7,0.1552,0.110857,NaN,https://www.statista.com/chart/21105/number-of...


<IPython.core.display.Javascript object>

In [25]:
beds = beds[beds.country != "Netherlands"]  # duplicated with "The Netherlands"
beds = beds[beds.country != "Dubai"]  # Dubai is missing data
beds = beds[beds.country != "Russia"]  # Russia is missing data
beds = beds[~beds.country.duplicated(keep="last")]  # Spain is there twice

<IPython.core.display.Javascript object>

In [26]:
rates = pd.read_csv("rates.txt")

<IPython.core.display.Javascript object>

In [27]:
rates

,Region,Hosp. rate,Crit rate,Fatality rate
0,WORLD,0.051678,0.013351,0.006457
1,UN development groups,0.122905,0.062122,0.029922
2,More developed regions,0.081276,0.026824,0.012962
3,Less developed regions,0.045899,0.010721,0.005187
4,Least developed countries,0.028594,0.005635,0.002727
...,...,...,...,...
250,Netherlands,0.083542,0.027369,0.013229
251,Switzerland,0.082858,0.027160,0.013125
252,NORTHERN AMERICA,0.073860,0.023118,0.011175
253,Canada,0.079015,0.025041,0.012104


<IPython.core.display.Javascript object>

This replacement needs to be done, so that the names of the regions correspond

In [28]:
to_replace = {
    "Czechia": "Czech Republic",
    "Brunei Darussalam": "Brunei",
    "United Arab Emirates": "Dubai",
    "China, Hong Kong SAR": "Hong Kong",
    "Lao People's Democratic Republic": "Laos",
    "China, Taiwan Province of China": "Taiwan",
    "Russian Federation": "Russia",
    "Netherlands": "The Netherlands",
    "United States of America": "United States",
}

<IPython.core.display.Javascript object>

In [29]:
rates.loc[:, "Region"] = rates.loc[:, "Region"].replace(to_replace)

<IPython.core.display.Javascript object>

In [30]:
countries_with_beds_and_rates = set(beds["country"]).intersection(set(rates["Region"]))

<IPython.core.display.Javascript object>

In [31]:
countries_with_beds_without_rates = set(beds["country"]).difference(
    set(rates["Region"])
)

<IPython.core.display.Javascript object>

In [32]:
countries_with_beds_without_rates

{'Andorra'}

<IPython.core.display.Javascript object>

In [33]:
beds_and_crit_rates = (
    pd.merge(
        beds.loc[:, ["country", "critical_beds_per_100k"]],
        rates.loc[:, ["Region", "Crit rate"]],
        how="left",
        left_on="country",
        right_on="Region",
    )
    .drop(columns=["Region"])
    .rename(columns={"Crit rate": "crit_rate"})
    .set_index("country")
)

<IPython.core.display.Javascript object>

In [34]:
beds_and_crit_rates[
    beds_and_crit_rates.loc[:, "crit_rate"].isna()
]  # only Andorra is missing critical region

,critical_beds_per_100k,crit_rate
country,,
Andorra,7.1,NaN


<IPython.core.display.Javascript object>

In [35]:
beds_and_crit_rates.loc[
    "Andorra", "crit_rate"
] = 0.026823769456994696  # "More developed regions" crit. rate

<IPython.core.display.Javascript object>

In [36]:
def max_percentage_infected_for_sufficient_icu_beds(df):
    return (df["critical_beds_per_100k"] / 1000) * (0.8 / df["crit_rate"])

<IPython.core.display.Javascript object>

In [37]:
final_data = beds_and_crit_rates.assign(
    max_percentage_infected_for_icu_beds=max_percentage_infected_for_sufficient_icu_beds,
)

<IPython.core.display.Javascript object>

In [38]:
final_data.to_csv("max_percentage_of_infected_population_for_enough_icu_beds.csv")

<IPython.core.display.Javascript object>

In [39]:
final_data[final_data.max_percentage_infected_for_icu_beds.isna()]

,critical_beds_per_100k,crit_rate,max_percentage_infected_for_icu_beds
country,,,


<IPython.core.display.Javascript object>